In [1]:
# importa de bibliotecas e módulos
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
# pd.options.display.max_colwidth = 500

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (20, 5),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# labels = {"city_st":"cidade (uf)",
#           "state":"estado",
#           "deaths":"mortes",
#           "confirmed":"casos confirmados",
#           "date":"date"}

import time
from datetime import date, datetime, timedelta
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

import unidecode

import json
from urllib.request import urlopen

import requests
from bs4 import BeautifulSoup

import funcoes_eda as ufm

In [2]:
# conda info --envs

In [3]:
# ufm.municipios_brio(uf)

In [7]:
today = date.today()
print(today)

now = str((datetime.now().strftime('%Y%m%d%H%M')))
print(now)

month = str((datetime.now().strftime('%m')))
# month = '03'
print(f"Mês: {month}")

str_date = str((datetime.now().strftime('%d%m')))
# str_date = '0703'
# print(str_date)

str_date2 = str((datetime.now().strftime('%d-%m')))
# str_date2 = '07-03'
print(str_date2)

uf = 'MA'
print(uf)

pdf = f'http://www.saude.ma.gov.br/wp-content/uploads/2021/{month}/BOLETIM-{str_date2}.pdf'
print(pdf)

csv_url = f'http://www.saude.ma.gov.br/wp-content/uploads/2021/{month}/Dados-Gerais-{str_date}.csv'
print(csv_url)
dados = pd.read_csv(csv_url, encoding='latin_1', sep=';', header=2, thousands='.')
display(dados.head())
dados.rename(columns=lambda x: unidecode.unidecode(x).strip().replace(" ", "_").lower(), inplace=True)

dados = dados[['municipios', 'confirmado', 'obito']]
dados.dropna(how='all', inplace=True)
dados.rename(columns={'municipios': 'municipio', 'confirmado': 'confirmados', 'obito': 'mortes'}, inplace=True)

# ufm.check_df(dados)
# ufm.check_null(dados)
# ufm.check_types(dados)

# display(dados.head())
# display(ufm.municipios_brio(uf).head())

df = dados.copy()


display(str_date, df.confirmados.sum(), df.mortes.sum())
df = df.replace({'Ignorado':'Importados/Indefinidos', 'Total': 'TOTAL NO ESTADO'})
# df = df.groupby(['municipio'], as_index=False).sum()
df['municipio'] = df['municipio'].str.replace('ITAPECURU-MIRIM', 'ITAPECURU MIRIM').str.replace('PERI-MIRIM', 'PERI MIRIM')
df['municipio'] = ufm.normaliza_txt_upper(df['municipio'])
display(df.shape, df.head())

df = pd.merge(df, ufm.municipios_brio(uf), on=['municipio'], how='outer')
df_sem_municipio = df[df['city'].isnull()]
display(f'SEM MATCH DE MUNICÍPIO:', df_sem_municipio)

df.drop(columns=['municipio'], inplace=True)
df.rename(columns={'city': 'municipio'}, inplace=True)
df = df[['municipio', 'confirmados', 'mortes']]

dict_add_rows = {'municipio':['TOTAL NO ESTADO',]
                     , 'confirmados': [df.confirmados.sum()]
                     , 'mortes': [df.mortes.sum()]}
df = pd.concat([pd.DataFrame(dict_add_rows), df]).reset_index(drop=True)

mask_tot = df.municipio == 'TOTAL NO ESTADO'
mask_imp = df.municipio == 'Importados/Indefinidos'

df = pd.concat([df[mask_tot], df[mask_imp], df[~mask_tot & ~mask_imp]], ignore_index=True)
mask_zeros = (df.confirmados == 0) & (df.mortes == 0)
df = df.dropna()[~mask_zeros]

df = pd.concat([pd.DataFrame({'municipio':['Importados/Indefinidos']
                     , 'confirmados': np.nan
                     , 'mortes': np.nan}), df]).reset_index(drop=True)

df[['confirmados', 'mortes']] = df[['confirmados', 'mortes']].astype("Int64")

display(df.shape)
ufm.check_null(df)

# SALVA ARQUIVO
now = str((datetime.now().strftime('%Y%m%d%H%M')))
filename = f'caso-{uf}-{str_date}_csv_ses.csv' 
df.to_csv(filename, index=False)
print(f"\nArquivo salvo: {filename}")

df.sort_values(by=['mortes', 'municipio'], ascending=[False, True])

display(df.shape, df)



2021-03-17
202103172101
Mês: 03
17-03
MA
http://www.saude.ma.gov.br/wp-content/uploads/2021/03/BOLETIM-17-03.pdf
http://www.saude.ma.gov.br/wp-content/uploads/2021/03/Dados-Gerais-1703.csv


,MUNICÍPIOS,CONFIRMADO,ÓBITO,Unnamed: 3,SUSPEITOS,DESCARTADOS,CONFIRMADOS,Unnamed: 7,LABORATÓRIO PÚBLICO,3.200
0,AÇAILÂNDIA,4193.0,131.0,NaN,1.749,431.521,231075,NaN,LABORATÓRIO PRIVADO,208
1,AFONSO CUNHA,154.0,2.0,NaN,NaN,NaN,NaN,NaN,BOLETIM ANTERIOR,560.986
2,ÁGUA DOCE DO MARANHÃO,665.0,7.0,NaN,CASOS CONFIRMADOS,NaN,NaN,NaN,TOTAL,564.394
3,ALCÂNTARA,203.0,8.0,NaN,STATUS,QUANTIDADE,NaN,NaN,NaN,NaN
4,ALDEIAS ALTAS,973.0,11.0,NaN,Ativos,12.944,Isolamento Domiciliar,11.326,NaN,NaN


'1703'

231075.0

5568.0

...

,municipio,confirmados,mortes,city


(219, 3)

Valores ausentes de confirmados: 1
Valores ausentes de mortes: 1

Arquivo salvo: caso-MA-1703_csv_ses.csv


(219, 3)

,municipio,confirmados,mortes
0,Importados/Indefinidos,<NA>,<NA>
1,TOTAL NO ESTADO,231075,5568
2,Açailândia,4193,131
3,Afonso Cunha,154,2
4,Água Doce do Maranhão,665,7
5,Alcântara,203,8
6,Aldeias Altas,973,11
7,Altamira do Maranhão,276,7
8,Alto Alegre do Maranhão,585,19
9,Alto Alegre do Pindaré,1684,26


### WEB SCRAPING
SCRAPER DA TABELA QUE ESTÁ NESTA PÁGINA:
https://www.corona.ma.gov.br/

In [19]:
req = requests.get('https://www.corona.ma.gov.br/')
if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content
    
soup = BeautifulSoup(content, 'html.parser')
table = soup.find(name='table')

table_str = str(table)
df_table = pd.read_html(table_str)[0]
df_table.rename(columns=lambda x: unidecode.unidecode(x).strip().replace(" ", "_").lower(), inplace=True)
df_table = df_table[['nome_do_municipio', 'confir-mados', 'obitos']]
ufm.check_df(df_table)

df_table.rename(columns={'nome_do_municipio': 'municipio', 'confir-mados': 'confirmados', 'obitos': 'mortes'}, inplace=True)
df_table.dropna(how='all', inplace=True)

# ufm.check_df(df_table)
# ufm.check_null(df_table)
# ufm.check_types(df_table)

# print(f"\n\nArquivo de {date_str}:")
# display(df_table.head())
# display(ufm.municipios_brio(uf).head())

df_table = df_table.copy()


display(str_date, df_table.confirmados.sum(), df_table.mortes.sum())
df_table = df_table.replace({'Ignorado':'Importados/Indefinidos', 'Total': 'TOTAL NO ESTADO'})
# df_table = df_table.groupby(['municipio'], as_index=False).sum()
df_table['municipio'] = df_table['municipio'].str.replace('ITAPECURU-MIRIM', 'ITAPECURU MIRIM').str.replace('PERI-MIRIM', 'PERI MIRIM')
df_table['municipio'] = ufm.normaliza_txt_upper(df_table['municipio'])
display(df_table.shape, df_table.head())

df_table = pd.merge(df_table, ufm.municipios_brio(uf), on=['municipio'], how='outer')
df_table_sem_municipio = df_table[df_table['city'].isnull()]
display(f'SEM MATCH DE MUNICÍPIO:', df_table_sem_municipio)

df_table.drop(columns=['municipio'], inplace=True)
df_table.rename(columns={'city': 'municipio'}, inplace=True)
df_table = df_table[['municipio', 'confirmados', 'mortes']]

dict_add_rows = {'municipio':['TOTAL NO ESTADO',]
                     , 'confirmados': [df_table.confirmados.sum()]
                     , 'mortes': [df_table.mortes.sum()]}
df_table = pd.concat([pd.DataFrame(dict_add_rows), df_table]).reset_index(drop=True)

mask_tot = df_table.municipio == 'TOTAL NO ESTADO'
mask_imp = df_table.municipio == 'Importados/Indefinidos'

df_table = pd.concat([df_table[mask_tot], df_table[mask_imp], df_table[~mask_tot & ~mask_imp]], ignore_index=True)
mask_zeros = (df_table.confirmados == 0) & (df_table.mortes == 0)
df_table = df_table.dropna()[~mask_zeros]

df_table = pd.concat([pd.DataFrame({'municipio':['Importados/Indefinidos']
                     , 'confirmados': np.nan
                     , 'mortes': np.nan}), df_table]).reset_index(drop=True)

df_table[['confirmados', 'mortes']] = df_table[['confirmados', 'mortes']].astype("Int64")

display(df_table.shape)
ufm.check_null(df_table)

# # SALVA ARQUIVO
# now = str((datetime.now().strftime('%Y%m%d%H%M')))
# filename = f'caso-{uf}-{str_date}_html_table_ses.csv' 
# df_table.to_csv(filename, index=False)
# print(f"\nArquivo salvo: {filename}")

df_table.sort_values(by=['mortes', 'municipio'], ascending=[False, True])

display(df_table.shape, df_table)



Requisição bem sucedida!

Total de 217 linhas e 3 colunas.

Colunas: ['nome_do_municipio', 'confir-mados', 'obitos']


,nome_do_municipio,confir-mados,obitos
0,AÇAILÂNDIA,3491,86


,nome_do_municipio,confir-mados,obitos
216,ZÉ DOCA,3766,38


'0111'

185827

...

,municipio,confirmados,mortes,city


(219, 3)

Valores ausentes de confirmados: 1
Valores ausentes de mortes: 1


(219, 3)

,municipio,confirmados,mortes
0,Importados/Indefinidos,<NA>,<NA>
1,TOTAL NO ESTADO,185827,4053
2,Açailândia,3491,86
3,Afonso Cunha,152,2
4,Água Doce do Maranhão,653,6
5,Alcântara,201,8
6,Aldeias Altas,939,5
7,Altamira do Maranhão,274,6
8,Alto Alegre do Maranhão,391,12
9,Alto Alegre do Pindaré,1488,23


### JSON

In [20]:
# DO JSON
start_url = 'https://www.corona.ma.gov.br/public/api/heatmap.json'
data = json.loads(
    
    
    urlopen(start_url).read())
# display(data)
df_json = pd.DataFrame(data)
df_json.rename(columns=lambda x: unidecode.unidecode(x).strip().replace(" ", "_").lower(), inplace=True)
df_json = df_json[['name', 'cases_c', 'deaths']]

df_json.rename(columns={'name': 'municipio', 'cases_c': 'confirmados', 'deaths': 'mortes'}, inplace=True)
df_json.dropna(how='all', inplace=True)
df_json[['confirmados', 'mortes']] = df_json[['confirmados', 'mortes']].astype(int)
ufm.check_df(df_json)
ufm.check_types(df_json)

# ufm.check_df(df_json)
# ufm.check_null(df_json)
# ufm.check_types(df_json)

# print(f"\n\nArquivo de {date_str}:")
# display(df_json.head())
# display(ufm.municipios_brio(uf).head())

df_json = df_json.copy()


display(str_date, df_json.confirmados.sum(), df_json.mortes.sum())
df_json = df_json.replace({'Ignorado':'Importados/Indefinidos', 'Total': 'TOTAL NO ESTADO'})
# df_json = df_json.groupby(['municipio'], as_index=False).sum()
df_json['municipio'] = df_json['municipio'].str.replace('ITAPECURU-MIRIM', 'ITAPECURU MIRIM').str.replace('PERI-MIRIM', 'PERI MIRIM')
df_json['municipio'] = ufm.normaliza_txt_upper(df_json['municipio'])
display(df_json.shape, df_json.head())

df_json = pd.merge(df_json, ufm.municipios_brio(uf), on=['municipio'], how='outer')
df_json_sem_municipio = df_json[df_json['city'].isnull()]
display(f'SEM MATCH DE MUNICÍPIO:', df_json_sem_municipio)

df_json.drop(columns=['municipio'], inplace=True)
df_json.rename(columns={'city': 'municipio'}, inplace=True)
df_json = df_json[['municipio', 'confirmados', 'mortes']]

dict_add_rows = {'municipio':['TOTAL NO ESTADO',]
                     , 'confirmados': [df_json.confirmados.sum()]
                     , 'mortes': [df_json.mortes.sum()]}
df_json = pd.concat([pd.DataFrame(dict_add_rows), df_json]).reset_index(drop=True)

mask_tot = df_json.municipio == 'TOTAL NO ESTADO'
mask_imp = df_json.municipio == 'Importados/Indefinidos'

df_json = pd.concat([df_json[mask_tot], df_json[mask_imp], df_json[~mask_tot & ~mask_imp]], ignore_index=True)
mask_zeros = (df_json.confirmados == 0) & (df_json.mortes == 0)
df_json = df_json.dropna()[~mask_zeros]

df_json = pd.concat([pd.DataFrame({'municipio':['Importados/Indefinidos']
                     , 'confirmados': np.nan
                     , 'mortes': np.nan}), df_json]).reset_index(drop=True)

df_json[['confirmados', 'mortes']] = df_json[['confirmados', 'mortes']].astype("Int64")

display(df_json.shape)
ufm.check_null(df_json)

# # SALVA ARQUIVO
# now = str((datetime.now().strftime('%Y%m%d%H%M')))
# filename = f'caso-{uf}-{str_date}_json_ses.csv' 
# df_json.to_csv(filename, index=False)
# print(f"\nArquivo salvo: {filename}")

df_json.sort_values(by=['mortes', 'municipio'], ascending=[False, True])

display(df_json.shape, df_json)




Total de 217 linhas e 3 colunas.

Colunas: ['municipio', 'confirmados', 'mortes']


,municipio,confirmados,mortes
0,São Luís,23396,1252


,municipio,confirmados,mortes
216,Amapá do Maranhão,4,0


coluna: municipio, tipo: <class 'str'>
coluna: confirmados, tipo: <class 'numpy.int64'>
coluna: mortes, tipo: <class 'numpy.int64'>


'0111'

...

,municipio,confirmados,mortes,city


(219, 3)

Valores ausentes de confirmados: 1
Valores ausentes de mortes: 1


(219, 3)

,municipio,confirmados,mortes
0,Importados/Indefinidos,<NA>,<NA>
1,TOTAL NO ESTADO,185827,4053
2,São Luís,23396,1252
3,Imperatriz,7751,385
4,Balsas,6510,68
5,Caxias,4977,110
6,Timon,4680,145
7,Santa Inês,4459,73
8,Barra do Corda,4031,42
9,Zé Doca,3766,37


In [21]:
print(f"\nCSV:\n")
ufm.check_df(df, 2, tail=False)
print(f"\nHTML TABLE:\n")
ufm.check_df(df_table, 2, tail=False)
print(f"\nJSON:\n")
ufm.check_df(df_json, 2, tail=False)


CSV:


Total de 219 linhas e 3 colunas.

Colunas: ['municipio', 'confirmados', 'mortes']


,municipio,confirmados,mortes
0,Importados/Indefinidos,<NA>,<NA>
1,TOTAL NO ESTADO,185917,4061



HTML TABLE:


Total de 219 linhas e 3 colunas.

Colunas: ['municipio', 'confirmados', 'mortes']


,municipio,confirmados,mortes
0,Importados/Indefinidos,<NA>,<NA>
1,TOTAL NO ESTADO,185827,4053



JSON:


Total de 219 linhas e 3 colunas.

Colunas: ['municipio', 'confirmados', 'mortes']


,municipio,confirmados,mortes
0,Importados/Indefinidos,<NA>,<NA>
1,TOTAL NO ESTADO,185827,4053
